In [1]:
import numpy as np
from functools import reduce

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

import DataPreprocessing
import SmartPolynomials
import time
import NLOPolynomials

4 Jets

In [2]:
combs = ['1,3','1,4','2,3','2,4','1,2,3','1,2,4','2,3,4','1,3,4','1,3','1,4','2,3','2,4','1,2,3','1,2,4','2,3,4','1,3,4']

In [3]:
##Obtain & process data NPY
#me_train, mom_train_r = DataPreprocessing.npy('LO/uuuxux/me_1000000.npy', 'LO/uuuxux/mom_1000000.npy', combs, frac=0.01)
me_test, mom_test = DataPreprocessing.npy('LO/uuuxux/me_500000.npy', 'LO/uuuxux/mom_500000.npy', combs, frac=1)

Matrix Element Prep in 1.5104 seconds


In [4]:
features = np.load('PolyCoef_uuuxuxLO.npy')
mean = np.load('mean_uuuxuxLO.npy')
std = np.load('std_uuuxuxLO.npy')
intercept = 4.904350663351414e+74

In [5]:
tic = time.perf_counter()

pred = np.zeros(len(me_test))
for i in range(0, len(me_test), 1000):
    if(i%50000==0): print(i)
    point = mom_test[i:i+1000]
    poly_test = SmartPolynomials.uuuxuxFeatures(point)
    
    poly_test = (poly_test - mean)/np.sqrt(std)
    
    temp = np.dot(poly_test, features) + intercept
    pred[i:i+1000] = temp
    
toc = time.perf_counter()

print(f"Ran in {toc - tic:0.4f} seconds")

0
50000
100000
150000
200000
250000
300000
350000
400000
450000
Ran in 215.5826 seconds


In [7]:
np.save('LO/uuuxux/pred_poly.npy', pred)

In [8]:
print(100*np.mean(np.abs((pred - me_test)/me_test)))

0.00016192638800066936


3 Jet NLO Finite

In [2]:
me_test, mom_test = DataPreprocessing.npy('NLO/uuxg/me_500000.npy', 'NLO/uuxg/mom_500000.npy', [], frac=1)
me_test = me_test[:,1]

tic = time.perf_counter()
div_test = reduce(np.multiply, DataPreprocessing.mandel_creation(['1,3','2,3'], mom_test))
temp_test = np.multiply(div_test, me_test)
toc = time.perf_counter()
print(f"Test multiplication ran in {toc - tic:0.4f} seconds")

Test multiplication ran in 0.2106 seconds


In [3]:
features = np.load('PolyCoef_uuxgfin.npy')
intercept = 498.85107759232767

In [4]:
tic = time.perf_counter()

pred = np.zeros(len(me_test))
for i in range(0, len(me_test), 5000):
    if(i%50000==0): print(i)
    point = mom_test[i:i+5000]
    poly_test = NLOPolynomials.uuxgFinFeatures(point)
    
    temp = np.dot(poly_test, features) + intercept
    pred[i:i+5000] = temp
    
toc = time.perf_counter()

print(f"Ran in {toc - tic:0.4f} seconds")

0
50000
100000
150000
200000
250000
300000
350000
400000
450000
Ran in 43.2983 seconds


In [5]:
print(100*np.mean(np.abs((pred - temp_test)/temp_test)))

0.0011201583728920911


In [6]:
np.save('NLO/uuxg/pred_poly_fin.npy', pred)